In [17]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from itertools import groupby
from operator import itemgetter
from CustomFunctions import DetailedBalance, utils

In [26]:
####### load common directories and data
time_interval = 10 #sec/frame
whichpcs = [1,7]
basedir = 'E:/Aaron/Combined_37C_Confocal_PCA/'
datadir = basedir + 'Data_and_Figs/'
FullFrame = pd.read_csv(datadir + 'All_Data_with_CGPS_bins.csv', index_col=0)
centers = pd.read_csv(datadir+'PC_bin_centers.csv', index_col=0)
nbins = np.max(FullFrame[[x for x in FullFrame.columns.to_list() if 'bin' in x]].to_numpy())
ttot = time_interval * 180

In [27]:
### restrict data to RANDOM
treatments = ['Random']

savedir = basedir + 'random/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#restrict dataframe to only random experiments
TotalFrame = FullFrame[FullFrame.Treatment=='Random'].copy()

In [15]:
### restrict data to PARANITROBLEBBISTATIN
treatments = ['DMSO','Para-Nitro-Blebbistatin']

savedir = basedir + 'Para-Nitro-Blebbistatin/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#limit data to the Para-Nitro-Blebbistatin experiments
TotalFrame = FullFrame[FullFrame.Experiment == 'Drug'].copy()
dates = [20240624,20240626,20240701,20241125,20241126,20241127]
TotalFrame = TotalFrame[TotalFrame.Date.isin(dates)]
TotalFrame['Treatment'] = pd.Categorical(TotalFrame.Treatment.to_list(), categories=treatments, ordered=True)

In [15]:
### restrict data to CK666
treatments = ['DMSO','CK666']

savedir = basedir + 'CK666/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

#limit data to the CK666 experiments
TotalFrame = FullFrame[FullFrame.Experiment == 'Drug'].copy()
dates = [20240610,20240617,20240620,20241205,20241209]
TotalFrame = TotalFrame[TotalFrame.Date.isin(dates)]
TotalFrame['Treatment'] = pd.Categorical(TotalFrame.Treatment.to_list(), categories=treatments, ordered=True)

In [28]:
###################

if __name__ ==  '__main__':
    ########### get raw transitions and pairs ###########
    rawtrans, rawpairs = DetailedBalance.get_raw_cgps_trajectories(
            TotalFrame, #pandas dataframe with all of the cgps binned data
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated trajectories
            )


    ########### interpolate all transitions so that only individual transitions are made ###########
    transdf_sep, transpairsdf_sep = DetailedBalance.get_interpolated_cgps_trajectories(
            TotalFrame, #pandas dataframe with all of the cgps binned data
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated trajectories
            )
    
    
    ############## get the counts of cells leaving 
    trans_rate_df_sep = DetailedBalance.aggregate_transition_counts(
            transdf_sep, #transdf_sep from get_interpolated_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            )

    ############## BOOTSTRAP MANY TRAJECTORIES ##########
    bstrans, bsint, bspairs, bsframe_sep_full = DetailedBalance.get_bootstrapped_cgps_trajectories(
            rawpairs, #raw transition pairs from get_raw_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            time_interval, #real time between datapoints
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            ttot, #set the total bootstrap time
            bsiter = 3000, #number of times to bootstrap
            )


    ############# open average bootstrapped currents ###################
    bsfield_sep = DetailedBalance.get_avg_current_error(
            bsframe_sep_full, #transition rates in the cgps from get_bootstrapped_cgps_trajectories
            whichpcs, #which two PCs to use in the cgps [x,y]
            savedir, #where to save the aggregated counts
            nbins, #how many bins in the x and y cgps axes
            )
    

Aggregated transitions
Finished interpolating trajectories
89144.60177165257
Finished finding transition rates
Starting Random trajectory 0-300
Starting Random trajectory 301-600
Starting Random trajectory 601-900
Starting Random trajectory 901-1200
Starting Random trajectory 1201-1500
Starting Random trajectory 1501-1800
Starting Random trajectory 1801-2100
Starting Random trajectory 2101-2400
Starting Random trajectory 2401-2700
Starting Random trajectory 2701-3000
Finished Random rates 0, starting 0-300
Finished Random rates 301, starting 301-600
Finished Random rates 601, starting 601-900
Finished Random rates 901, starting 901-1200
Finished Random rates 1201, starting 1201-1500
Finished Random rates 1501, starting 1501-1800
Finished Random rates 1801, starting 1801-2100
Finished Random rates 2101, starting 2101-2400
Finished Random rates 2401, starting 2401-2700
Finished Random rates 2701, starting 2701-3000
Finished bootstrapping


In [29]:
################ get aer and cfs ##################
if __name__ ==  '__main__':
    if os.path.exists(savedir+f'PC{whichpcs[0]}-PC{whichpcs[1]}_bootstrapped_transitions.csv'):
        bstrans = pd.read_csv(savedir+f'PC{whichpcs[0]}-PC{whichpcs[1]}_bootstrapped_transitions.csv', index_col=0)
        xyscaling = [centers[f'PC{whichpcs[0]}'].diff().mean(),centers[f'PC{whichpcs[1]}'].diff().mean()]
        center = [7,7]
        DetailedBalance.get_aer_cf(
            bstrans, #boostrapped transitions from get_bootstrapped_cgps_trajectories
            nbins, #how many bins in the x and y cgps axes
            xyscaling, #scaling of the bins in real units of whatever the CGPS axis parameters are
            center, #origin in [x bin,y bin]
            savedir, #where to save calculated aers and cfs
            whichpcs, #which two PCs to use in the cgps [x,y]
            )

Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2600
Finished 2601, starting 2601-2800
Finished 2801, starting 2801-3000


In [11]:
############# create all CGPSs #############

binlist = [i for i in TotalFrame.columns.to_list() if 'bin' in i]
allsavedir = savedir + 'allCGPS/'
if not os.path.exists(allsavedir):
    os.makedirs(allsavedir)
for a in range(1,len(binlist)+1):
    for b in range(1,len(binlist)+1):
        if a == b:
            continue
        elif os.path.exists(allsavedir+f'interpolated_PC{b}-PC{a}_transitions_separated.csv'):
            print('Already made this plot')
            continue
        else:
            #set the PCs
            abwhichpcs = [a,b]
            if __name__ ==  '__main__':
                
                ########### get raw transitions and pairs ###########
                rawtrans, rawpairs = DetailedBalance.get_raw_cgps_trajectories(
                        TotalFrame, #pandas dataframe with all of the cgps binned data
                        abwhichpcs, #which two PCs to use in the cgps [x,y]
                        time_interval, #real time between datapoints
                        allsavedir, #where to save the aggregated trajectories
                        )
                
                ########### interpolate all transitions so that only individual transitions are made ###########
                transdf_sep, transpairsdf_sep = DetailedBalance.get_interpolated_cgps_trajectories(
                        TotalFrame, #pandas dataframe with all of the cgps binned data
                        abwhichpcs, #which two PCs to use in the cgps [x,y]
                        time_interval, #real time between datapoints
                        allsavedir, #where to save the aggregated trajectories
                        )

                ############## get the counts of cells leaving 
                trans_rate_df_sep = DetailedBalance.aggregate_transition_counts(
                        transdf_sep, #transdf_sep from get_interpolated_cgps_trajectories
                        abwhichpcs, #which two PCs to use in the cgps [x,y]
                        allsavedir, #where to save the aggregated counts
                        nbins, #how many bins in the x and y cgps axes
                        )
                
                
                
                ############# measure aer and cycling frequencies ###########
                #add specific scaling
                xyscaling = [centers[f'PC{whichpcs[0]}'].diff().mean(),centers[f'PC{whichpcs[1]}'].diff().mean()]
                #set the origin to the actual center
                center = [round(nbins/2)]*2
                #add "iter" so that I can use get_aer_cf
                itertrans = []
                itern = 0
                for i, cells in rawtrans.groupby('CellID'):
                    cells, runs = utils.get_consecutive_timepoints(cells, 'time',time_interval)
                    for r in runs:
                        cell = cells.iloc[r]
                        cell['iter'] = itern
                        itern = itern+1
                        itertrans.append(cell)
                itertrans = pd.concat(itertrans)

                DetailedBalance.get_aer_cf(
                    itertrans,
                    nbins, #how many bins in the x and y cgps axes
                    xyscaling, #scaling of the bins in real units of whatever the CGPS axis parameters are
                    center, #origin in [x bin,y bin]
                    allsavedir, #where to save calculated aers and cfs
                    abwhichpcs, #which two PCs to use in the cgps [x,y]
                    )

Aggregated transitions
Finished interpolating trajectories
167573.25067924493
190137.445336424
Finished finding transition rates


C:\ProgramData\anaconda3\envs\abhishape\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Aggregated transitions
Finished interpolating trajectories
168341.34895432135
191043.87340766555
Finished finding transition rates
Finished 0, starting 

Finished 2401, starting 2401-2598
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Already made this plot
Aggregated transitions
Finished interpolating trajectories
168684.44604745795
190800.87388556555
Finished finding transition rates
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-240

Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Already made this plot
Already made this plot
Aggregated transitions
Finished interpolating trajectories
168599.0402293978
191482.84977006825
Finished finding transition rates
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1

Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Aggregated transitions
Finished interpolating trajectories
168889.60650982324
191338.8679839182
Finished finding transition rates
Finished 0, starting 0-200
Finished 201, starting

Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Aggregated transitions
Finished interpolating trajectories
169239.80853695417
192437.71964128833
Finished finding transition rates
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starti

Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1801-2000
Finished 2001, starting 2001-2200
Finished 2201, starting 2201-2400
Finished 2401, starting 2401-2598
Already made this plot
Already made this plot
Already made this plot
Already made this plot
Already made this plot
Already made this plot
Already made this plot
Aggregated transitions
Finished interpolating trajectories
169461.324470305
191967.27080589934
Finished finding transition rates
Finished 0, starting 0-200
Finished 201, starting 201-400
Finished 401, starting 401-600
Finished 601, starting 601-800
Finished 801, starting 801-1000
Finished 1001, starting 1001-1200
Finished 1201, starting 1201-1400
Finished 1401, starting 1401-1600
Finished 1601, starting 1601-1800
Finished 1801, starting 1

In [5]:
bin1, bin2

('PC1', 'PC2')